# Instalar polar para cargar las bases de pangenomas

In [ ]:
# pip install polar

# Librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import polars as pl
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [ ]:
# Lee el archivo TSV
df_polar = pl.read_csv("PangenomeCountCiprofloxacin.tsv", separator='\t')

# Muestra el DataFrame
df_polar

In [ ]:
# Convertir a pandas
df = df_polar.to_pandas()
df

# Todos los datos

In [3]:
# Define your algorithms
algorithms = {
        "Random Forest (1200 trees)": RandomForestClassifier(n_estimators=1200, random_state=42),
        "Random Forest (500 trees)": RandomForestClassifier(n_estimators=500, random_state=42),
        "KNN (k=1)": KNeighborsClassifier(n_neighbors=1),
        "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
        "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
        "SVM (RBF Kernel)": SVC(kernel='rbf', gamma='auto', C=1, random_state=42),
        "SVM (Linear Kernel)": SVC(kernel='linear', gamma='auto', C=1, random_state=42),
        "SVM (Polynomial Kernel)": SVC(kernel='poly', gamma='auto', C=1, random_state=42),
        "MLP (1 Hidden Layer, 200 neurons)": MLPClassifier(hidden_layer_sizes=(200,), max_iter=1000, random_state=42),
        "Logistic Regression": LogisticRegression(random_state=42),
        "Gaussian NB": GaussianNB()
}
# Train and evaluate each algorithm
results = {}
for algo_name, algo in algorithms.items():
    # Train the model
    # Filtra el DataFrame por la columna 'phenotype'
    #df_training = df[df['phenotype'].isin(['Susceptible', 'Resistant']) & df['genus'].isin(['Your_Genus_Filter'])]
    df_training = df[df['phenotype'].isin(['Susceptible', 'Resistant'])]
    df_test = df[df['phenotype'].isna()]

    # Filtra el DataFrame por cada etiqueta
    df_res = df[df['phenotype'] == 'Resistant']
    df_sus = df[df['phenotype'] == 'Susceptible']

    # Calcula la proporción de 1 y 0 para cada etiqueta en cada columna
    proporcion_res = df_res.iloc[:,6:].mean()
    proporcion_sus = df_sus.iloc[:,6:].mean()

    # Calcula la diferencia en la proporción entre las dos etiquetas para cada columna
    diferencia = abs(proporcion_res - proporcion_sus)

    # Ordena las columnas por la diferencia en orden descendente
    columnas_ordenadas = diferencia.sort_values(ascending=False)

    # Selecciona las N primeras columnas con la mayor diferencia
    # OBSERVACION: FUE UN NUMERO ALEATORIO SIN SENTIDO
    columns_subset_size = 4000
    subset = columnas_ordenadas.head(columns_subset_size)

    nombres_columnas_seleccionadas = subset.index

    # Filtra el DataFrame original para seleccionar solo las columnas deseadas
    df_filtrado = df_training[nombres_columnas_seleccionadas]
    df_filtrado_test = df_test[nombres_columnas_seleccionadas]

    labels = df_training['phenotype']
    label_encoder = LabelEncoder()

    # Fit and transform the labels
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Dividir los datos codificados en conjunto de entrenamiento y conjunto de prueba
    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(df_filtrado, encoded_labels, stratify=encoded_labels, test_size=test_size, random_state=50)
    
       
    algo.fit(X_train, y_train)

    # Make predictions
    y_pred = algo.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted F1-score for multiclass classification
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results[algo_name] = {'accuracy': accuracy, 'f1_score': f1, 'confusion_matrix': cm}
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta verdadera')
    plt.xticks(ticks=[0.5, 1.5], labels=['Resistant', 'Susceptible'])
    plt.yticks(ticks=[0.5, 1.5], labels=['Resistant', 'Susceptible'])
    plt.title('Matriz de Confusión - ' + algo_name)
    
    # Save confusion matrix plot
    output_folder = "confusion_matrix_plots"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.savefig(os.path.join(output_folder, algo_name + "_confusion_matrix.png"))
    plt.close()

# Save results to CSV
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("results.csv")

# Print results
for algo_name, result in results.items():
    print(f"Algorithm: {algo_name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"F1 Score: {result['f1_score']:.4f}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


/opt/tljh/user/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Algorithm: Random Forest (1200 trees)
Accuracy: 0.9590
F1 Score: 0.9526
Confusion Matrix:
[[ 20  22]
 [  1 518]]

Algorithm: Random Forest (500 trees)
Accuracy: 0.9590
F1 Score: 0.9526
Confusion Matrix:
[[ 20  22]
 [  1 518]]

Algorithm: KNN (k=1)
Accuracy: 0.9501
F1 Score: 0.9446
Confusion Matrix:
[[ 20  22]
 [  6 513]]

Algorithm: KNN (k=3)
Accuracy: 0.9394
F1 Score: 0.9400
Confusion Matrix:
[[ 26  16]
 [ 18 501]]

Algorithm: KNN (k=5)
Accuracy: 0.9554
F1 Score: 0.9536
Confusion Matrix:
[[ 26  16]
 [  9 510]]

Algorithm: SVM (RBF Kernel)
Accuracy: 0.9447
F1 Score: 0.9424
Confusion Matrix:
[[ 23  19]
 [ 12 507]]

Algorithm: SVM (Linear Kernel)
Accuracy: 0.9483
F1 Score: 0.9431
Confusion Matrix:
[[ 20  22]
 [  7 512]]

Algorithm: SVM (Polynomial Kernel)
Accuracy: 0.9269
F1 Score: 0.8935
Confusion Matrix:
[[  1  41]
 [  0 519]]

Algorithm: MLP (1 Hidden Layer, 200 neurons)
Accuracy: 0.9483
F1 Score: 0.9439
Confusion Matrix:
[[ 21  21]
 [  8 511]]

Algorithm: Logistic Regression
Accuracy

# Campylobacter y Neisseria

In [4]:
# Define your algorithms
algorithms = {
        "Random Forest (1200 trees)": RandomForestClassifier(n_estimators=1200, random_state=42),
        "Random Forest (500 trees)": RandomForestClassifier(n_estimators=500, random_state=42),
        "KNN (k=1)": KNeighborsClassifier(n_neighbors=1),
        "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
        "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
        "SVM (RBF Kernel)": SVC(kernel='rbf', gamma='auto', C=1, random_state=42),
        "SVM (Linear Kernel)": SVC(kernel='linear', gamma='auto', C=1, random_state=42),
        "SVM (Polynomial Kernel)": SVC(kernel='poly', gamma='auto', C=1, random_state=42),
        "MLP (1 Hidden Layer, 200 neurons)": MLPClassifier(hidden_layer_sizes=(200,), max_iter=1000, random_state=42),
        "Logistic Regression": LogisticRegression(random_state=42),
        "Gaussian NB": GaussianNB()
    }

# Train and evaluate each algorithm
results = {}
for algo_name, algo in algorithms.items():
    # Train the model
    # Filtra el DataFrame por la columna 'phenotype'
    df_training = df[df['phenotype'].isin(['Susceptible', 'Resistant']) & df['genus'].isin(['Campylobacter', 'Neisseria'])]
    #df_training = df[df['phenotype'].isin(['Susceptible', 'Resistant'])]
    df_test = df[df['phenotype'].isna()& df['genus'].isin(['Campylobacter', 'Neisseria', 'Escherichia'])]

    # Filtra el DataFrame por cada etiqueta
    df_res = df[df['phenotype'] == 'Resistant']
    df_sus = df[df['phenotype'] == 'Susceptible']

    # Calcula la proporción de 1 y 0 para cada etiqueta en cada columna
    proporcion_res = df_res.iloc[:,6:].mean()
    proporcion_sus = df_sus.iloc[:,6:].mean()

    # Calcula la diferencia en la proporción entre las dos etiquetas para cada columna
    diferencia = abs(proporcion_res - proporcion_sus)

    # Ordena las columnas por la diferencia en orden descendente
    columnas_ordenadas = diferencia.sort_values(ascending=False)

    # Selecciona las N primeras columnas con la mayor diferencia
    columns_subset_size = 4000
    subset = columnas_ordenadas.head(columns_subset_size)

    nombres_columnas_seleccionadas = subset.index

    # Filtra el DataFrame original para seleccionar solo las columnas deseadas
    df_filtrado = df_training[nombres_columnas_seleccionadas]
    #df_filtrado_test = df_test[nombres_columnas_seleccionadas]

    labels = df_training['phenotype']
    label_encoder = LabelEncoder()

    # Fit and transform the labels
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Dividir los datos codificados en conjunto de entrenamiento y conjunto de prueba
    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(df_filtrado, encoded_labels, stratify=encoded_labels, test_size=test_size, random_state=50)
    
    #X_train = df.drop(columns=['phenotype'])  # Assuming 'phenotype' is the target column
    #y_train = df['phenotype']
    
    algo.fit(X_train, y_train)

    # Make predictions
    y_pred = algo.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted F1-score for multiclass classification
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results[algo_name] = {'accuracy': accuracy, 'f1_score': f1, 'confusion_matrix': cm}
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta verdadera')
    plt.xticks(ticks=[0.5, 1.5], labels=['Resistant', 'Susceptible'])
    plt.yticks(ticks=[0.5, 1.5], labels=['Resistant', 'Susceptible'])
    plt.title('Matriz de Confusión - ' + algo_name)
    
    # Save confusion matrix plot
    output_folder = "confusion_matrix_plots_CNE"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.savefig(os.path.join(output_folder, algo_name + "_confusion_matrix.png"))
    plt.close()

# Save results to CSV
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("results_CNE.csv")

# Print results
for algo_name, result in results.items():
    print(f"Algorithm: {algo_name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"F1 Score: {result['f1_score']:.4f}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


/opt/tljh/user/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Algorithm: Random Forest (1200 trees)
Accuracy: 0.8308
F1 Score: 0.8231
Confusion Matrix:
[[24 16]
 [ 6 84]]

Algorithm: Random Forest (500 trees)
Accuracy: 0.8385
F1 Score: 0.8302
Confusion Matrix:
[[24 16]
 [ 5 85]]

Algorithm: KNN (k=1)
Accuracy: 0.8000
F1 Score: 0.8025
Confusion Matrix:
[[29 11]
 [15 75]]

Algorithm: KNN (k=3)
Accuracy: 0.8154
F1 Score: 0.8109
Confusion Matrix:
[[25 15]
 [ 9 81]]

Algorithm: KNN (k=5)
Accuracy: 0.7769
F1 Score: 0.7791
Confusion Matrix:
[[27 13]
 [16 74]]

Algorithm: SVM (RBF Kernel)
Accuracy: 0.7769
F1 Score: 0.7704
Confusion Matrix:
[[22 18]
 [11 79]]

Algorithm: SVM (Linear Kernel)
Accuracy: 0.8308
F1 Score: 0.8189
Confusion Matrix:
[[22 18]
 [ 4 86]]

Algorithm: SVM (Polynomial Kernel)
Accuracy: 0.7000
F1 Score: 0.5840
Confusion Matrix:
[[ 1 39]
 [ 0 90]]

Algorithm: MLP (1 Hidden Layer, 200 neurons)
Accuracy: 0.8308
F1 Score: 0.8189
Confusion Matrix:
[[22 18]
 [ 4 86]]

Algorithm: Logistic Regression
Accuracy: 0.8231
F1 Score: 0.8095
Confusion 

# Salmonella

In [5]:
# Define your algorithms
algorithms = {
        "Random Forest (1200 trees)": RandomForestClassifier(n_estimators=1200, random_state=42),
        "Random Forest (500 trees)": RandomForestClassifier(n_estimators=500, random_state=42),
        "KNN (k=1)": KNeighborsClassifier(n_neighbors=1),
        "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
        "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
        "SVM (RBF Kernel)": SVC(kernel='rbf', gamma='auto', C=1, random_state=42),
        "SVM (Linear Kernel)": SVC(kernel='linear', gamma='auto', C=1, random_state=42),
        "SVM (Polynomial Kernel)": SVC(kernel='poly', gamma='auto', C=1, random_state=42),
        "MLP (1 Hidden Layer, 200 neurons)": MLPClassifier(hidden_layer_sizes=(200,), max_iter=1000, random_state=42),
        "Logistic Regression": LogisticRegression(random_state=42),
        "Gaussian NB": GaussianNB()
    }

# Train and evaluate each algorithm
results = {}
for algo_name, algo in algorithms.items():
    # Train the model
    # Filtra el DataFrame por la columna 'phenotype'
    df_training = df[df['phenotype'].isin(['Susceptible', 'Resistant']) & df['genus'].isin(['Salmonella'])]
    #df_training = df[df['phenotype'].isin(['Susceptible', 'Resistant'])]
    df_test = df[df['phenotype'].isna()& df['genus'].isin(['Salmonella'])]

    # Filtra el DataFrame por cada etiqueta
    df_res = df[df['phenotype'] == 'Resistant']
    df_sus = df[df['phenotype'] == 'Susceptible']

    # Calcula la proporción de 1 y 0 para cada etiqueta en cada columna
    proporcion_res = df_res.iloc[:,6:].mean()
    proporcion_sus = df_sus.iloc[:,6:].mean()

    # Calcula la diferencia en la proporción entre las dos etiquetas para cada columna
    diferencia = abs(proporcion_res - proporcion_sus)

    # Ordena las columnas por la diferencia en orden descendente
    columnas_ordenadas = diferencia.sort_values(ascending=False)

    # Selecciona las N primeras columnas con la mayor diferencia
    columns_subset_size = 5000
    subset = columnas_ordenadas.head(columns_subset_size)

    nombres_columnas_seleccionadas = subset.index

    # Filtra el DataFrame original para seleccionar solo las columnas deseadas
    df_filtrado = df_training[nombres_columnas_seleccionadas]
    #df_filtrado_test = df_test[nombres_columnas_seleccionadas]

    labels = df_training['phenotype']
    label_encoder = LabelEncoder()

    # Fit and transform the labels
    encoded_labels = label_encoder.fit_transform(labels)
    
    # Dividir los datos codificados en conjunto de entrenamiento y conjunto de prueba
    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(df_filtrado, encoded_labels, stratify=encoded_labels, test_size=test_size, random_state=50)
    
    #X_train = df.drop(columns=['phenotype'])  # Assuming 'phenotype' is the target column
    #y_train = df['phenotype']
    
    algo.fit(X_train, y_train)

    # Make predictions
    y_pred = algo.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted F1-score for multiclass classification
    cm = confusion_matrix(y_test, y_pred)

    # Store results
    results[algo_name] = {'accuracy': accuracy, 'f1_score': f1, 'confusion_matrix': cm}
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta verdadera')
    plt.xticks(ticks=[0.5, 1.5], labels=['Resistant', 'Susceptible'])
    plt.yticks(ticks=[0.5, 1.5], labels=['Resistant', 'Susceptible'])
    plt.title('Matriz de Confusión - ' + algo_name)
    
    # Save confusion matrix plot
    output_folder = "confusion_matrix_plots_Salmonella"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.savefig(os.path.join(output_folder, algo_name + "_confusion_matrix.png"))
    plt.close()

# Save results to CSV
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("results_Salmonella.csv")

# Print results
for algo_name, result in results.items():
    print(f"Algorithm: {algo_name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"F1 Score: {result['f1_score']:.4f}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


/opt/tljh/user/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Algorithm: Random Forest (1200 trees)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: Random Forest (500 trees)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: KNN (k=1)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: KNN (k=3)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: KNN (k=5)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: SVM (RBF Kernel)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: SVM (Linear Kernel)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: SVM (Polynomial Kernel)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: MLP (1 Hidden Layer, 200 neurons)
Accuracy: 0.9954
F1 Score: 0.9931
Confusion Matrix:
[[  0   2]
 [  0 430]]

Algorithm: Logistic Regression
Accuracy